In [ ]:
import numpy as np
from sklearn import datasets

import elastic.container
from elastic.record_event import RecordEvent

In [ ]:
@RecordEvent
def cell_2():
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    
    return iris_X, iris_y

In [ ]:
iris_X, iris_y = cell_2()

In [ ]:
@RecordEvent
def cell_3():
    np.random.seed(0)

In [ ]:
cell_3()

In [ ]:
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test

In [ ]:
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

@RecordEvent
def cell_5():
    knn = KNeighborsClassifier()
    knn.fit(iris_X_train, iris_y_train)

    knn.predict(iris_X_test)
    
    return knn

In [ ]:
knn = cell_5()

In [ ]:
for x in elastic.container.operation_events:
    print(x, "\n")

In [ ]:
for x in elastic.container.data_events:
    print(x, "\n")

In [ ]:
parent_lookup = {oe: set() for oe in elastic.container.operation_events}
children_lookup = {oe: set() for oe in elastic.container.operation_events}

In [ ]:
for oe in elastic.container.operation_events:
    for related_data_event in elastic.container.operation_event_lookup[oe.exec_uuid].related_data_events:
        parent_oe = related_data_event.prev_operation_event
        
        parent_lookup[oe].add(parent_oe)
        children_lookup[parent_oe].add(oe)